In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lgbm
#from xgboost import XGBRegressor
import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
pd.set_option('display.max_columns', 500)
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn import metrics

import sys, os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import math
import pickle
import mplcyberpunk
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback, EarlyStopping


In [2]:
from itertools import islice, cycle

plt.style.use("cyberpunk")

def add_secondary_plot(df, column, target_column, ax, n_bins, color=3, show_yticks=False, marker="."):
    secondary_ax = ax.twinx()
    bins = pd.cut(df[column], bins=n_bins)
    bins = pd.IntervalIndex(bins)
    bins = (bins.left + bins.right) / 2
    target = df.groupby(bins)[target_column].mean()
    target.plot(
        ax=secondary_ax, linestyle='',
        marker=marker, color=color, label=f"Mean '{target_column}'"
    )
    secondary_ax.grid(visible=False)
    
    if not show_yticks:
        secondary_ax.get_yaxis().set_ticks([])
        
    return secondary_ax

def render_feature_distros(train_df, test_df, features=[], labels=[], n_bins=50, n_cols=4, pad=2, h_pad=4, w_pad=None):
    histplot_hyperparams = {
        'kde':True,
        'alpha':0.4,
        'stat':'percent',
        'bins':n_bins
    }
    markers = ['.', '+', 'x', '1', '2']
    
    n_rows = math.ceil(len(features) / n_cols)
    cell_with_dim = 4
    cell_height_dim = 3
    
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))
    plt.tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=None)
    
    # delete exess subplots
    for a in ax[n_rows - 1, int(((n_rows - (len(features) / n_cols)) * n_cols*-1)):]:
        a.axis('off')
        
    leg_handles = []
    leg_labels = []
    
    axs = []

    for i, feature in enumerate(features):
        row = math.ceil(i / n_cols) - 1
        col = (i % n_cols)
        
        color_cycle = islice(mplcyberpunk.cyberpunk_stylesheets['cyberpunk']['axes.prop_cycle'], 0, None)
        
        sns.histplot(train_df[feature], label='Train X', ax=ax[row, col], color=next(color_cycle)['color'], **histplot_hyperparams)
        sns.histplot(test_df[feature], label='Test X', ax=ax[row, col],color=next(color_cycle)['color'], **histplot_hyperparams)
        ax[row, col].set_title(f'{feature} Distribution')
        mplcyberpunk.make_lines_glow(ax[row, col])
        axs.append(ax[row, col].get_legend_handles_labels())

        for j, label in enumerate(labels):
            sub_ax = add_secondary_plot(train_df, feature, label, ax[row, col], n_bins, color=next(color_cycle)['color'], marker=markers[j])
            axs.append(sub_ax.get_legend_handles_labels())
        
    for axis in axs:
        if axis[1][0] not in leg_labels:
            leg_labels.extend(axis[1])
            leg_handles.extend(axis[0])
        
    fig.legend(leg_handles, leg_labels, loc='upper center', bbox_to_anchor=(0.5, 1.04), fontsize=14, ncol=len(features) + 2)

In [20]:
train_time = 1 * 60 * 60
objective = 'tfregressor'

In [4]:
sys.path.append(os.path.realpath('..'))

In [5]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [6]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
Age,0,0.0
StockOptionLevel,0,0.0
Over18,0,0.0
OverTime,0,0.0
PercentSalaryHike,0,0.0
PerformanceRating,0,0.0
RelationshipSatisfaction,0,0.0
StandardHours,0,0.0
TotalWorkingYears,0,0.0
BusinessTravel,0,0.0


In [7]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [8]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [9]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [10]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [11]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}

import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)


def objective_v3(trial):
    oof = np.zeros(len(X))
    scores = []
   
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        X_train, X_valid, y_train, y_valid = train_test_split(train_df[features], train_df[target], test_size=0.20)

        horsepower_model = tf.keras.Sequential([
            layers.Dense(1000, activation="relu"),
            layers.Dropout(0.1),
            layers.Dense(1000, activation="relu"),
            layers.Dense(1, activation="sigmoid"),
        ])

        #reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        horsepower_model.compile(
            loss='binary_crossentropy', optimizer='adam',metrics=['accuracy']
        )

        history = horsepower_model.fit(
            X_train,
            y_train,
            validation_data=(X_valid, y_valid),

            epochs=10,
            # Suppress logging.
            verbose=1,
        )
           
        oof[valid_idx] = horsepower_model.predict(X_valid)[0]

    roc_auc = roc_auc_score(Y, oof)
    
    param_grid_history[roc_auc] = {}

    return roc_auc

In [32]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v3, timeout=train_time)

[I 2023-01-21 21:21:57,090] A new study created in memory with name: xgbregressor


========== Fold=1 ==========
Epoch 1/10
42/42 [==============================] - 1s 11ms/step - loss: 315.9835 - accuracy: 0.8113 - val_loss: 69.5992 - val_accuracy: 0.7024
Epoch 2/10
42/42 [==============================] - 0s 3ms/step - loss: 35.5689 - accuracy: 0.8046 - val_loss: 82.1756 - val_accuracy: 0.8631
Epoch 3/10
42/42 [==============================] - 0s 3ms/step - loss: 24.7279 - accuracy: 0.8143 - val_loss: 22.1791 - val_accuracy: 0.8423
Epoch 4/10
42/42 [==============================] - 0s 3ms/step - loss: 28.1157 - accuracy: 0.8039 - val_loss: 18.4902 - val_accuracy: 0.8631
Epoch 5/10
42/42 [==============================] - 0s 3ms/step - loss: 23.2843 - accuracy: 0.8098 - val_loss: 7.6418 - val_accuracy: 0.8661
Epoch 6/10
42/42 [==============================] - 0s 3ms/step - loss: 13.3962 - accuracy: 0.8128 - val_loss: 14.5156 - val_accuracy: 0.8631
Epoch 7/10
42/42 [==============================] - 0s 3ms/step - loss: 12.1440 - accuracy: 0.8106 - val_loss: 9.6673 

[W 2023-01-21 21:22:02,360] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_15160\3881561340.py", line 37, in objective_v3
    history = horsepower_model.fit(
  File "D:\source\repos\venv\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\keras\engine\training.py", line 1606, in fit
    val_logs = self.evaluate(
  File "D:\source\repos\venv\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\keras\engine\training.py", line 1947, in evaluate
    tmp_logs = self.test_function(iterator)
  File "D:\sour

KeyboardInterrupt: 

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df[features], train_df[target], test_size=0.20)
horsepower_model = tf.keras.Sequential([
    layers.Dense(1000, activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(1000, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])

#reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
horsepower_model.compile(
    loss='binary_crossentropy', optimizer='adam',metrics=['accuracy']
)


history = horsepower_model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),

    epochs=100,
    # Suppress logging.
    verbose=1,
)

horsepower_model.predict(X_valid)[0]

Epoch 1/100
42/42 [==============================] - 1s 6ms/step - loss: 393.1921 - accuracy: 0.7942 - val_loss: 11.0909 - val_accuracy: 0.6250
Epoch 2/100
42/42 [==============================] - 0s 3ms/step - loss: 30.7881 - accuracy: 0.7860 - val_loss: 16.1888 - val_accuracy: 0.7708
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 20.3969 - accuracy: 0.7875 - val_loss: 4.6991 - val_accuracy: 0.7857
Epoch 4/100
42/42 [==============================] - 0s 3ms/step - loss: 20.8214 - accuracy: 0.7793 - val_loss: 17.9223 - val_accuracy: 0.8899
Epoch 5/100
42/42 [==============================] - 0s 3ms/step - loss: 14.0799 - accuracy: 0.8061 - val_loss: 3.7521 - val_accuracy: 0.8869
Epoch 6/100
42/42 [==============================] - 0s 3ms/step - loss: 7.3073 - accuracy: 0.7897 - val_loss: 6.6587 - val_accuracy: 0.8571
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 7.3433 - accuracy: 0.7987 - val_loss: 3.7537 - val_accuracy: 0.7292
Epoc

array([0.70306814], dtype=float32)

In [23]:
objective

'tfregressor'

In [35]:
train_preds = []
train_preds.append(horsepower_model.predict(train_df[features]))
train_final_preds = np.stack(train_preds).mean(0)

train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds.reshape(len(train_final_preds)).tolist()})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

53/53 [==============================] - 0s 1ms/step


In [37]:
test_preds = []
test_preds.append(horsepower_model.predict(test_df[features]))
test_final_preds = np.stack(test_preds).mean(0)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds.reshape(len(test_final_preds)).tolist()})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)

35/35 [==============================] - 0s 1000us/step


In [33]:
train_final_preds.reshape(len(train_final_preds)).tolist()

[0.29763534665107727,
 0.2145659178495407,
 0.14717520773410797,
 0.1498907506465912,
 0.4386022388935089,
 0.29969337582588196,
 0.5491160750389099,
 0.17812363803386688,
 0.08284812420606613,
 0.2441578060388565,
 0.20705869793891907,
 0.27838873863220215,
 0.34607991576194763,
 0.3098334074020386,
 0.21122094988822937,
 0.27135735750198364,
 0.265879362821579,
 0.13833941519260406,
 0.1397162824869156,
 0.11753559857606888,
 0.4148927628993988,
 0.269684761762619,
 0.41575735807418823,
 0.2872975468635559,
 0.21741734445095062,
 0.20274662971496582,
 0.3006517291069031,
 0.23984336853027344,
 0.2692815959453583,
 0.20518939197063446,
 0.0895732119679451,
 0.07171088457107544,
 0.23958809673786163,
 0.0762794092297554,
 0.17831914126873016,
 0.3704150319099426,
 0.39833006262779236,
 0.22744838893413544,
 0.11358622461557388,
 0.8305570483207703,
 0.3068463206291199,
 0.24102213978767395,
 0.40601611137390137,
 0.307915598154068,
 0.198457270860672,
 0.07826673239469528,
 0.347584605

In [ ]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [ ]:
def train(param_grid):
    reg = xgb.XGBModel(
        # These parameters should help with trial speed.
        objective='binary:logistic',
        tree_method='gpu_hist',
        booster='gbtree',
        predictor='gpu_predictor',
        n_jobs=4,
        eval_metric='auc',
        early_stopping_rounds=100,
        **param_grid
    )
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.20)
    
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

In [35]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)

NameError: name 'train_final_preds' is not defined